# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime
from matplotlib.ticker import MaxNLocator

from functions import train_model

In [2]:
# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [ ]:
portfolio_raw

In [4]:
#portfolio_raw['duration']*24

In [ ]:
portfolio_raw.info()

In [ ]:
portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [ ]:
profile_raw.head()

In [ ]:
profile_raw.info()

In [ ]:
profile_raw.describe(include='all')

In [ ]:
profile_raw['gender'].value_counts(dropna=False)

In [ ]:
profile_raw[profile_raw['gender'].isna()]['age'].value_counts(dropna=False)

In [ ]:
profile_raw[profile_raw['gender'].isna()]['income'].value_counts(dropna=False)

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [ ]:
transcript_raw.head()

In [ ]:
transcript_raw.info()

In [ ]:
transcript_raw['value']

In [ ]:
transcript_raw['event'].unique()

In [ ]:
set([list(x.keys())[0] for x in transcript_raw['value']])

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [13]:
#storage profiles with noneType gender values in separeted files
#profile_none = profile_raw[profile_raw['gender'].isna()].copy()
#profile = profile_raw[~profile_raw['gender'].isna()].copy()
profile = profile_raw.copy()




# changing the id GUIDs (Globally Unique Identifiers) to short tag.
port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}



portfolio = portfolio_raw.copy()
portfolio['id_short'] = portfolio_raw['id'].map(port_id)

# Standardisation of column names
transcript = transcript_raw.copy()
transcript['event'] = transcript_raw['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)


# creating a dataframe handling the data in 'value' column
value_df = pd.json_normalize(transcript['value']).fillna(0)
value_df['offer_id'] = value_df['offer_id'].map(port_id).fillna('tran')
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

# merging the transcript and profile dataframes
transcript_b = transcript_b.merge(profile_raw, left_on='person', right_on='id', how='left')

# mergin transcript and portfolio dataframes
transcript_b = transcript_b.merge(portfolio, left_on='offer_id', right_on='id_short', how='left', suffixes=('_x', '_std'))

# dropping unnecessary columns
transcript_b = transcript_b.drop(['id_x', 'reward_x', 'id_std', 'id_short'], axis=1)

transcript_b['became_member_on'] = pd.to_datetime(transcript_b['became_member_on'], format='%Y%m%d')
transcript_b['bec_memb_month'] = pd.to_datetime(transcript_b['became_member_on'], format='%Y%m%d').dt.strftime('%m')
transcript_b['bec_memb_day'] = pd.to_datetime(transcript_b['became_member_on'], format='%Y%m%d').dt.strftime('%d')
transcript_b['bec_memb_year_month'] = pd.to_datetime(transcript_b['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')



#removing 'transactions' from the transcript_b dataframe
transactions = transcript_b.loc[transcript_b['event'] == 'transaction',:].dropna(axis=1)
transcript_gender_null = transcript_b.loc[(transcript_b['gender'].isna() & (transcript_b['event'] != 'transaction')),:]
transcript_b = transcript_b.loc[(~transcript_b['gender'].isna()) & (transcript_b['event'] != 'transaction'),:]



def categorize_age(age):
    if age < 25:
        return 'Young_Adult'
    elif 25 <= age < 40:
        return 'Adult'
    elif 40 <= age < 60:
        return 'Middle_Aged'
    else:
        return 'Senior'
    
transcript_b['age_group'] = transcript_b['age'].apply(categorize_age)

transcript_b.to_csv('transcript_b.csv')

transcript_b

,person,event,time,offer_id,amount,gender,age,became_member_on,income,reward_std,channels,difficulty,duration,offer_type,bec_memb_month,bec_memb_day,bec_memb_year_month,age_group
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,ofr_D,0.0,F,75,2017-05-09,100000.0,5.0,"[web, email, mobile]",5.0,7.0,bogo,05,09,2017-05,Senior
2,e2127556f4f64592b11af22de27a7932,offer_received,0,ofr_J,0.0,M,68,2018-04-26,70000.0,2.0,"[web, email, mobile]",10.0,7.0,discount,04,26,2018-04,Senior
5,389bc3fa690240e798340f5a15918d5c,offer_received,0,ofr_I,0.0,M,65,2018-02-09,53000.0,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,02,09,2018-02,Senior
7,2eeac8d8feae4a8cad5a6af0499a211d,offer_received,0,ofr_C,0.0,M,58,2017-11-11,51000.0,0.0,"[web, email, mobile]",0.0,4.0,informational,11,11,2017-11,Middle_Aged
8,aa4862eba776480b8bb9c68455b8c2e1,offer_received,0,ofr_E,0.0,F,61,2017-09-11,57000.0,5.0,"[web, email]",20.0,10.0,discount,09,11,2017-09,Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306483,84fb57a7fe8045a8bf6236738ee73a0f,offer_viewed,714,ofr_H,0.0,F,64,2017-06-15,113000.0,0.0,"[email, mobile, social]",0.0,3.0,informational,06,15,2017-06,Senior
306490,abc4359eb34e4e2ca2349da2ddf771b6,offer_viewed,714,ofr_C,0.0,M,51,2014-08-04,68000.0,0.0,"[web, email, mobile]",0.0,4.0,informational,08,04,2014-08,Middle_Aged
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer_viewed,714,ofr_E,0.0,F,60,2017-09-08,64000.0,5.0,"[web, email]",20.0,10.0,discount,09,08,2017-09,Senior
306509,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,ofr_G,0.0,M,39,2018-06-27,39000.0,2.0,"[web, email, mobile, social]",10.0,10.0,discount,06,27,2018-06,Adult


In [7]:
transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148805 entries, 0 to 306527
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   person               148805 non-null  object        
 1   event                148805 non-null  object        
 2   time                 148805 non-null  int64         
 3   offer_id             148805 non-null  object        
 4   amount               148805 non-null  float64       
 5   gender               148805 non-null  object        
 6   age                  148805 non-null  int64         
 7   became_member_on     148805 non-null  datetime64[ns]
 8   income               148805 non-null  float64       
 9   reward_std           148805 non-null  float64       
 10  channels             148805 non-null  object        
 11  difficulty           148805 non-null  float64       
 12  duration             148805 non-null  float64       
 13  offer_type         

In [12]:
transcript_b.columns.tolist()

['person',
 'event',
 'time',
 'offer_id',
 'amount',
 'gender',
 'age',
 'became_member_on',
 'income',
 'reward_std',
 'channels',
 'difficulty',
 'duration',
 'offer_type',
 'bec_memb_month',
 'bec_memb_day',
 'bec_memb_year_month',
 'age_group']

In [ ]:
transactions.info()

In [ ]:
transactions.info()

In [ ]:
transcript_b['tag'] = (
    transcript_b.groupby(['person', 'event', 'offer_id'])
    .cumcount()
)

person_event = transcript_b.groupby(['person', 'event', 'gender', 'age', 'offer_type', 'income'], dropna=False).agg(
    cnt = ('event', 'count'),
    t = ('time', 'mean'),
    #age = ('age', 'max'),
    
    ).unstack([1]).reset_index() #.fillna(np.inf)

person_event.columns = ['_'.join(col).strip('_') for col in person_event.columns.to_flat_index()]


person_event['to_vr'] = person_event['t_offer_viewed'] - person_event['t_offer_received']
person_event['to_cv'] = person_event['t_offer_completed'] - person_event['t_offer_viewed'] 
person_event['to_cr'] = person_event['t_offer_completed'] - person_event['t_offer_received']

person_event['curiosity_vr'] = (1 / (person_event['to_vr'] + 1)) * 100
person_event['eagerness_cv'] = (1 / ((person_event['to_cv']) + 1)) * 100
person_event['overall_cr'] = (1 / (person_event['to_cr'] + 1)) * 100

person_event['influence'] = ((person_event['t_offer_completed'] != 1000) & (person_event['t_offer_completed'] >= person_event['t_offer_viewed'])).astype(int) 
person_event['ext_influence'] = ((person_event['t_offer_viewed'] != 1000) & (person_event['t_offer_completed'] == person_event['t_offer_viewed'])).astype(int) 


df0 = person_event

for c in person_event.select_dtypes('number').columns:
    df0[c] = df0[c].replace(np.inf, np.nan).fillna(0).astype(int)

df0 = df0.drop(columns=['t_offer_completed', 't_offer_viewed', 't_offer_received'])
df0 = df0.drop(columns=['to_vr', 'to_cv', 'to_cr'])
df0 = df0.round(1)





df0.to_csv('person_activity_features.csv')

person_event.to_csv('person_event_features.csv')


df0

,person,gender,age,offer_type,income,cnt_offer_completed,cnt_offer_received,cnt_offer_viewed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence,age_group
0,0009655768c64bdeb2e877511632db8f,M,33,bogo,72000,1,1,1,2,-2,14,0,0,Adult
1,0009655768c64bdeb2e877511632db8f,M,33,discount,72000,2,2,1,100,7,7,1,0,Adult
2,0009655768c64bdeb2e877511632db8f,M,33,informational,72000,0,2,2,3,0,0,0,0,Adult
3,0011e0d4e6b944f998e987f904e8c1e5,O,40,bogo,57000,1,1,1,7,1,1,1,0,Middle_Aged
4,0011e0d4e6b944f998e987f904e8c1e5,O,40,discount,57000,2,2,2,4,0,0,1,0,Middle_Aged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,informational,56000,0,1,0,0,0,0,0,0,Senior
35320,fffad4f4828548d1b5583907f2e9906b,M,34,bogo,34000,3,3,3,1,-7,1,0,0,Adult
35321,fffad4f4828548d1b5583907f2e9906b,M,34,informational,34000,0,1,1,100,0,0,0,0,Adult
35322,ffff82501cea40309d5fdd7edcca4a07,F,45,bogo,62000,1,1,1,3,-3,100,0,0,Middle_Aged


In [8]:
person_event

,person,offer_id,gender,age,offer_type,income,cnt_offer_completed,cnt_offer_received,cnt_offer_viewed,t_offer_completed,t_offer_received,t_offer_viewed,to_vr,to_cv,to_cr,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence
0,0009655768c64bdeb2e877511632db8f,ofr_C,M,33,informational,72000.0,0,1,1,NaN,336.0,372.0,36.0,NaN,NaN,2.702703,NaN,NaN,0,0
1,0009655768c64bdeb2e877511632db8f,ofr_G,M,33,discount,72000.0,1,1,1,528.0,504.0,540.0,36.0,-12.0,24.0,2.702703,-9.090909,4.000000,0,0
2,0009655768c64bdeb2e877511632db8f,ofr_H,M,33,informational,72000.0,0,1,1,NaN,168.0,192.0,24.0,NaN,NaN,4.000000,NaN,NaN,0,0
3,0009655768c64bdeb2e877511632db8f,ofr_I,M,33,bogo,72000.0,1,1,1,414.0,408.0,456.0,48.0,-42.0,6.0,2.040816,-2.439024,14.285714,0,0
4,0009655768c64bdeb2e877511632db8f,ofr_J,M,33,discount,72000.0,1,1,0,576.0,576.0,NaN,NaN,NaN,0.0,NaN,NaN,100.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55217,fffad4f4828548d1b5583907f2e9906b,ofr_I,M,34,bogo,34000.0,2,2,2,276.0,204.0,258.0,54.0,18.0,72.0,1.818182,5.263158,1.369863,1,0
55218,ffff82501cea40309d5fdd7edcca4a07,ofr_D,F,45,bogo,62000.0,1,1,1,504.0,504.0,534.0,30.0,-30.0,0.0,3.225806,-3.448276,100.000000,0,0
55219,ffff82501cea40309d5fdd7edcca4a07,ofr_E,F,45,discount,62000.0,1,1,1,198.0,168.0,174.0,6.0,24.0,30.0,14.285714,4.000000,3.225806,1,0
55220,ffff82501cea40309d5fdd7edcca4a07,ofr_G,F,45,discount,62000.0,1,1,1,60.0,0.0,6.0,6.0,54.0,60.0,14.285714,1.818182,1.639344,1,0


In [9]:
person_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55222 entries, 0 to 55221
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               55222 non-null  object 
 1   offer_id             55222 non-null  object 
 2   gender               55222 non-null  object 
 3   age                  55222 non-null  int64  
 4   offer_type           55222 non-null  object 
 5   income               55222 non-null  float64
 6   cnt_offer_completed  55222 non-null  int64  
 7   cnt_offer_received   55222 non-null  int64  
 8   cnt_offer_viewed     55222 non-null  int64  
 9   t_offer_completed    27942 non-null  float64
 10  t_offer_received     55222 non-null  float64
 11  t_offer_viewed       42493 non-null  float64
 12  to_vr                42493 non-null  float64
 13  to_cv                23499 non-null  float64
 14  to_cr                27942 non-null  float64
 15  curiosity_vr         42493 non-null 

In [ ]:
person_event['gender'].value_counts(dropna=False)

In [5]:
event_age = transcript_b.groupby(['person','offer_id', 'gender', 'event', 'age', 'income']).agg(
    eve_cnt = ('event', 'count'),
    avg_t = ('time', 'mean'),
    #avg_age = ('age', 'mean'),
    
    ).unstack([3]).round(1).reset_index()

event_age.columns = ['_'.join(col).strip('_') for col in event_age.columns.to_flat_index()]

event_age['com_rate'] = event_age.eve_cnt_offer_completed / event_age.eve_cnt_offer_received
event_age['view_rate'] = event_age.eve_cnt_offer_viewed / event_age.eve_cnt_offer_received
event_age['avg_t_diff']  = event_age.avg_t_offer_viewed - event_age.avg_t_offer_received

event_age = event_age.fillna(0)

event_age.to_csv('event_age_offer_type.csv')

event_age


,person,offer_id,gender,age,income,eve_cnt_offer_completed,eve_cnt_offer_received,eve_cnt_offer_viewed,avg_t_offer_completed,avg_t_offer_received,avg_t_offer_viewed,com_rate,view_rate,avg_t_diff
0,0009655768c64bdeb2e877511632db8f,ofr_C,M,33,72000.0,0.0,1.0,1.0,0.0,336.0,372.0,0.0,1.0,36.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,M,33,72000.0,1.0,1.0,1.0,528.0,504.0,540.0,1.0,1.0,36.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,M,33,72000.0,0.0,1.0,1.0,0.0,168.0,192.0,0.0,1.0,24.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,M,33,72000.0,1.0,1.0,1.0,414.0,408.0,456.0,1.0,1.0,48.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,M,33,72000.0,1.0,1.0,0.0,576.0,576.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55217,fffad4f4828548d1b5583907f2e9906b,ofr_I,M,34,34000.0,2.0,2.0,2.0,276.0,204.0,258.0,1.0,1.0,54.0
55218,ffff82501cea40309d5fdd7edcca4a07,ofr_D,F,45,62000.0,1.0,1.0,1.0,504.0,504.0,534.0,1.0,1.0,30.0
55219,ffff82501cea40309d5fdd7edcca4a07,ofr_E,F,45,62000.0,1.0,1.0,1.0,198.0,168.0,174.0,1.0,1.0,6.0
55220,ffff82501cea40309d5fdd7edcca4a07,ofr_G,F,45,62000.0,1.0,1.0,1.0,60.0,0.0,6.0,1.0,1.0,6.0


In [ ]:
sns.scatterplot(data=event_age, x='eve_cnt_offer_received', y='com_rate', hue='gender', palette='Set2', size='offer_id') 	#
plt.tight_layout()

In [ ]:
sns.scatterplot(data=event_age[event_age['gender'] == 'M'], x='view_rate', y='com_rate', marker='x') #, hue='offer_id', palette='Set2'
sns.scatterplot(data=event_age[event_age['gender'] == 'F'], x='view_rate', y='com_rate', marker='o')
sns.scatterplot(data=event_age[event_age['gender'] == 'O'], x='view_rate', y='com_rate', marker='*') #, , hue='offer_id'size='offer_id', palette='Set2'
plt.legend(title='Gender', labels=['Male', 'Female', 'O'])
plt.title('Offer received vs. Completion rate')
plt.xlabel('Viewed rate')   
plt.ylabel('Completion rate')

#plt.xlim(2500, 4000)
plt.ylim(0, 1)

In [ ]:
event_age[event_age['offer_id'] == 'ofr_A']

In [ ]:
sns.scatterplot(
    data=event_age,
    x='view_rate',
    y='com_rate',
    hue='offer_id',  
    
)

plt.title('Offer Viewed vs. Completion rate')
plt.xlabel('Viewed rate')   
plt.ylabel('Completion rate')

plt.xlim(0,1.1)
plt.ylim(0,1.1)

In [ ]:
sns.scatterplot(
    data=event_age,
    y='view_rate',
    x='com_rate',
    hue='offer_id',
    size='gender',  
    sizes=(20, 200),
    legend='full',
)

# Configurar o título e rótulos
plt.title('Offer Viewed vs. Completion Rate (Bubble Size = Gender)')
plt.xlabel('Viewed Rate')
plt.ylabel('Completion Rate')
plt.xlim(0, 1.1)
plt.ylim(0, 1.1)

# Ajustar a posição da legenda
plt.legend(
    bbox_to_anchor=(1.05, 1),  # Posição da legenda (à direita e acima)
    loc='upper left',         # Alinha no canto superior esquerdo da âncora
    borderaxespad=0           # Remove o espaçamento extra entre o gráfico e a legenda
)

plt.tight_layout()  # Ajusta o layout para evitar sobreposição
plt.show()

In [ ]:
sns.pairplot(
    data=event_age, #[event_age['offer_id'] == 'ofr_G']
    vars=['view_rate', 'com_rate', 'avg_t_diff'],  # Substitua pelas métricas desejadas
    hue='gender',
    palette='Set2',
    diag_kind='kde',
)

plt.suptitle('Pairplot: Relationships Between Variables', y=1.02)
plt.show()

In [ ]:
event_age[event_age['offer_id'] == 'ofr_G']

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(9,6))  

sns.scatterplot(data=event_age[event_age['offer_id'] == 'ofr_G'], x='com_rate', y='avg_t_diff', ax=axes[0,0], hue='gender')
sns.scatterplot(data=event_age[event_age['offer_id'] == 'ofr_A'], x='com_rate', y='avg_t_diff', ax=axes[0,1], hue='gender')
sns.scatterplot(data=event_age[event_age['offer_id'] == 'ofr_I'], x='com_rate', y='avg_t_diff', ax=axes[1,0], hue='gender')
sns.scatterplot(data=event_age[event_age['offer_id'] == 'ofr_B'], x='com_rate', y='avg_t_diff', ax=axes[1,1], hue='gender')


axes[0,0].set_title('offer_id_G')
axes[0,0].set_xlabel('')
axes[0,0].set_ylabel('avg_t_diff')
axes[0,0].set_ylim(10, 30)
axes[0,0].set_xlim(0, 1) 
axes[0,0].legend('') 
axes[0,0].set_xticks([])

axes[0,1].set_title('offer_id_A')
axes[0,1].set_xlabel('')
axes[0,1].set_ylabel('')
axes[0,1].set_ylim(10, 30)
axes[0,1].set_xlim(0, 1)  
axes[0,1].set_xticks([])

axes[1,0].set_title('offer_id_I')
axes[1,0].set_xlabel('com_rate')
axes[1,0].set_ylabel('avg_t_diff')
axes[1,0].set_ylim(10, 30)
axes[1,0].set_xlim(0, 1)  
axes[1,0].legend('') 


axes[1,1].set_title('offer_id_B')
axes[1,1].set_xlabel('com_rate')
axes[1,1].set_ylabel('')
axes[1,1].set_ylim(10, 30)
axes[1,1].set_xlim(0, 1)  
axes[1,1].legend('') 

plt.show()


In [ ]:
sns.boxplot(data=event_age, y='com_rate', x='offer_id') #, inner='quartile'

In [ ]:
sns.boxplot(data=event_age, y='view_rate', x='offer_id')

In [ ]:
sns.histplot(data=event_age, x='avg_age_offer_completed', hue='gender', bins=10, kde=True) #[event_age['offer_id'] == 'ofr_A']

In [ ]:
sns.histplot(data=event_age[event_age['offer_id'] == 'ofr_G'], x='avg_age_offer_completed', hue='gender', bins=10, kde=True, weights='avg_age_offer_completed')

In [ ]:
offer_event = transcript_b.groupby(['offer_id', 'gender', 'event'])['event'].count().unstack(2).reset_index()

offer_event['com_rate'] = offer_event['offer_completed'] / offer_event['offer_received']
offer_event['vie_rate'] = offer_event['offer_viewed'] / offer_event['offer_received']


offer_event

In [ ]:
sns.histplot(data=transcript_b, x='became_member_on')

In [ ]:
time_base = transcript_b.groupby(['bec_memb_year_month', 'gender']).agg(
    person_count=('person', 'count'),
).reset_index()

time_base['person_count_cumsum'] = time_base.groupby('gender')['person_count'].cumsum()

gender_age_base = transcript_b.loc[:, ['gender', 'age']]


fig, axes = plt.subplots(3, 3, figsize=(10, 8))  

sns.histplot(data=time_base[time_base['gender'] == 'M'], x='bec_memb_year_month',weights='person_count',  bins=10, color='lightblue',ax=axes[0,0])
sns.histplot(data=time_base[time_base['gender'] == 'F'],x='bec_memb_year_month',weights='person_count',  bins=10,  color='pink',ax=axes[1,0])
sns.histplot(data=time_base[time_base['gender'] == 'O'],x='bec_memb_year_month',weights='person_count',  bins=10,  color='grey',ax=axes[2,0])

sns.histplot(data=time_base[time_base['gender'] == 'M'], x='bec_memb_year_month',weights='person_count_cumsum',  bins=10, color='lightblue',ax=axes[0,1])
sns.histplot(data=time_base[time_base['gender'] == 'F'],x='bec_memb_year_month',weights='person_count_cumsum',  bins=10,  color='pink',ax=axes[1,1])
sns.histplot(data=time_base[time_base['gender'] == 'O'],x='bec_memb_year_month',weights='person_count_cumsum',  bins=10,  color='grey',ax=axes[2,1])

sns.histplot(data=gender_age_base[gender_age_base['gender'] == 'M']['age'], bins=30, color='lightblue', ax=axes[0,2])
sns.histplot(data=gender_age_base[gender_age_base['gender'] == 'F']['age'], bins=30, color='pink', ax=axes[1,2])
sns.histplot(data=gender_age_base[gender_age_base['gender'] == 'O']['age'], bins=30, color='grey', ax=axes[2,2])

axes[0,0].set_title('Became Member (Histogram)')
axes[0,0].set_xlabel('')
axes[0,0].set_ylabel('Custumers / Year-Month')
axes[0,0].set_xticks([])
axes[0,0].set_ylim(0, 4500)  


axes[1,0].set_title('')
axes[1,0].set_xlabel('')
axes[1,0].set_ylabel('Custumers / Year-Month')
axes[1,0].set_xticks([])
axes[1,0].set_ylim(0, 4500)  


axes[2,0].set_title('')
axes[2,0].set_xlabel('')
axes[2,0].set_ylabel('Custumers / Year-Month')
axes[2,0].set_xticks([])
axes[2,0].set_ylim(0, 1000)  
axes[2,0].xaxis.set_major_locator(MaxNLocator(integer=True, prune='both', nbins=10))
axes[2,0].tick_params(axis='x', rotation=45)


axes[0,1].set_title('Became Member (Cumulative sum)')
axes[0,1].set_xlabel('')
axes[0,1].set_ylabel('')
axes[0,1].set_xticks([])
axes[0,1].set_ylim(0, 100000)  


axes[1,1].set_title('')
axes[1,1].set_xlabel('')
axes[1,1].set_ylabel('')
axes[1,1].set_xticks([])
axes[1,1].set_ylim(0, 100000)  


axes[2,1].set_title('')
axes[2,1].set_xlabel('')
axes[2,1].set_ylabel('')
#axes[2,1].set_xticks([])
axes[2,1].set_ylim(0, 5000)  
axes[2,1].xaxis.set_major_locator(MaxNLocator(integer=True, prune='both', nbins=10))
axes[2,1].tick_params(axis='x', rotation=45)


axes[0,2].set_title('Age (Histogram)')
axes[0,2].set_xlabel('')
axes[0,2].set_ylabel('')
axes[0,2].set_xticks([])
axes[0,2].set_ylim(0, 6000)  
axes[0,2].set_xlim(0, 120)  


axes[1,2].set_title('')
axes[1,2].set_xlabel('')
axes[1,2].set_ylabel('')
axes[1,2].set_xticks([])
axes[1,2].set_ylim(0, 6000)  
axes[1,2].set_xlim(0, 120)  

axes[2,2].set_title('')
axes[2,2].set_xlabel('')
axes[2,2].set_ylabel('')
#axes[2,2].set_xticks([])
axes[2,2].set_ylim(0, 1000)
axes[2,2].set_xlim(0, 120)    


# Ajustar layout para evitar sobreposição
plt.tight_layout()
plt.show()

#sns.barplot(data=time_base, y=time_base['person_count'], hue='bec_memb_year_month', palette='Set2')

In [ ]:
transcript_b.loc[:, ['gender', 'age']]

In [ ]:
time_base['person'].cumsum()

In [ ]:
# creating 'tag' column
transcript_b['tag'] = (
    transcript_b.groupby(['person', 'event', 'offer_id'])
    .cumcount()
)

(
transcript_b.pivot_table(
    index=['person', 'offer_id', 'tag'], 
    columns='event',
    values='time', 
    margins=False, 
    dropna=True).reset_index()
    
)


In [ ]:
transcript_b.info()

In [ ]:
transactions

In [ ]:
# cheking profile 
profile.describe(include='all').round(0) 

In [ ]:
profile.info()

In [ ]:
# Checking the data transcript_b created
transcript_b.info()

# Grouping/Merging Datasets and Features Engineering

**task:**
### Creating the `person_activity` Table: Customer Interaction Insights  

The goal is to construct the `person_activity` table to analyze customer behavior by tracking their event timelines and generating relevant time-based features.

This structured dataset enables detailed analysis of how customers interact with offers, their responsiveness, and purchasing behaviors over time.  

---

#### **Objective**  
Track and organize customer interactions (`person`, `event`, `offer_id`) into a structured `timeline`, generating features that offer insight into customer behavior.  

---

#### **Steps**  

1. **Create the `tag` column:**  
   - Enumerate unique combinations of `['person', 'event', 'offer_id']` to identify distinct events.  

2. **Group and unstack:**  
   - Group data by `person`, `offer_id`, and `tag`, and unstack the event timeline to create a structured timeline for customer interactions.  

3. **Merge Transaction Data:**  
   - Include transactional data by linking `time` and `amount` fields with the timeline structure.  

4. **Generate Count-Based Features:**  
   - Count key events (`offer_received`, `offer_viewed`, `offer_completed`) for each customer and offer.  
   - Add features to measure transaction-related behaviors such as the number of transactions.

5. **Calculate Time-Based Features:**  
   - Compute the time differences between events (`offer_received`, `offer_viewed`, and `offer_completed`) to assess responsiveness and engagement.  

6. **Behavioral Metrics:**  
   - Develop derived metrics to measure curiosity, eagerness, and overall responsiveness using the time differences.  

7. **Influence Indicators:**  
   - Identify whether customers were influenced by the offers based on the relationship between event timestamps.  

8. **Additional Features and Cleaning:**  
   - Compute average engagement time and handle missing values for transaction-related features like `amount` and `number of transactions`.  

---

#### **Outcome**  

The `person_activity` table will contain:  

1. **Timeline Events** for each `person` and `offer_id`:  
   - Key columns include `offer_received`, `offer_viewed`, and `offer_completed`.  

### Derived Features  

1. **Count-Based Features:**  
   - `n_rec`: Number of received offers.  
   - `n_vie`: Number of viewed offers.  
   - `n_com`: Number of completed offers.  
   - `n_tran`: Number of transactions.  
   - `amt_mean`: Average transaction amount.  

2. **Time-Based Metrics:**  
   - `to_vr`: Time between viewing and receiving an offer.  
   - `to_cv`: Time between completing and viewing an offer.  
   - `to_cr`: Time between completing and receiving an offer.  

3. **Behavioral Metrics:**  
   - `curiosity_vr`: Metric reflecting curiosity (based on time to view).  
   - `eagerness_cv`: Metric reflecting eagerness (based on time to complete).  
   - `overall_cr`: Overall responsiveness (based on time to complete after receiving).  

4. **Influence Indicators:**  
   - `ofr_influence`: Indicates if completion happened after viewing.  
   - `extrm_ofr_influence`: Indicates if completion occurred immediately after viewing.  



In [ ]:
# creating 'tag' column
transcript_b['tag'] = (
    transcript_b.groupby(['person', 'event', 'offer_id'])
    .cumcount()
)


# Group and unstack the transcript_b dataframe
person_activity = (
    transcript_b[transcript_b['event'] != 'transaction']
    .groupby(['person', 'offer_id', 'tag', 'event'])['time']
    .first()
    .unstack() #fill_value=5000
    .reset_index()
)

# Filtering the transcript_b dataframe to only include transactions
transactions = transcript_b.loc[transcript_b['event'] == 'transaction', ['person', 'time', 'amount']]


# Merge the person_activity dataframe with the transactions dataframe
person_activity = pd.merge(
    how='left',
    left=person_activity,
    right=transactions,
    left_on=['person', 'offer_completed'],
    right_on=['person', 'time'],
).drop(columns=['time'])


# Creating count features using the groupby() function and the agg() function. Merge the result with the person_activity dataframe
aggregations1 = person_activity.groupby(['person', 'offer_id']).agg(
    n_rec=('offer_received', 'count'),
    n_vie=('offer_viewed', 'count'),
    n_com=('offer_completed', 'count'),

).reset_index()

person_activity = person_activity.merge(aggregations1, on=['person', 'offer_id'], how='left')


aggregations2 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['person', 'offer_id']).agg(
    n_tran = ('event', 'count'),
    amt_mean = ('amount', 'mean')

)

person_activity = person_activity.merge(aggregations2, on=['person'], how='left')


# Creating other features directly from the person_activity dataframe

person_activity['to_vr'] = person_activity['offer_viewed'] - person_activity['offer_received']
person_activity['to_cv'] = person_activity['offer_completed'] - person_activity['offer_viewed'] 
person_activity['to_cr'] = person_activity['offer_completed'] - person_activity['offer_received']

person_activity['curiosity_vr'] = (1 / (person_activity['to_vr'] + 1)) * 100
person_activity['eagerness_cv'] = (1 / (person_activity['to_cv'] + 1)) * 100
person_activity['overall_cr'] = (1 / (person_activity['to_cr'] + 1)) * 100

person_activity['ofr_influence'] = ((person_activity['offer_completed'] != 1000) & (person_activity['offer_completed'] >= person_activity['offer_viewed'])) 
person_activity['extrm_ofr_influence'] = ((person_activity['offer_viewed'] != 1000) & (person_activity['offer_completed'] == person_activity['offer_viewed']))

person_activity['avg_time_to_engage'] = ((person_activity['to_vr'] + person_activity['to_cv']) / 2)


person_activity.loc[:,'amount'] = person_activity.loc[:,'amount'].fillna(0)
person_activity.loc[:,'n_tran'] = person_activity.loc[:,'n_tran'].fillna(0)
person_activity.loc[:,'amt_mean'] = person_activity.loc[:,'amt_mean'].fillna(0)

person_activity = person_activity.round(2)

person_activity

In [ ]:
person_activity.info()

In [ ]:
person_activity[person_activity['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

In [ ]:
# Group and unstack the transcript_b dataframe
offer_activity = (
    transcript_b[transcript_b['event'] != 'transaction']
    .groupby(['offer_id', 'event'])['time'].mean()
    .unstack() #fill_value=5000
    .reset_index()
)

offer_activity

In [ ]:
transcript_b

In [ ]:
profile

In [ ]:
transcript_gen = transcript_b.merge(profile, left_on='person', right_on='id', how='left')
transcript_gen = transcript_gen.drop(columns=['id'])

gender_activity = (
    transcript_gen[transcript_gen['event'] != 'transaction']
    .groupby(['gender', 'event'])['time'].mean()
    .unstack() #fill_value=5000
    .reset_index()
)

gender_activity['to_rv'] = gender_activity['offer_viewed'] - gender_activity['offer_received']
gender_activity['to_vc'] = gender_activity['offer_completed'] - gender_activity['offer_viewed']
gender_activity['to_rc'] = gender_activity['offer_completed'] - gender_activity['offer_received']


gender_activity.round(1)

In [ ]:
age_activity = (
    transcript_gen[transcript_gen['event'] != 'transaction']
    .groupby(['age', 'event'])['time'].mean()
    .unstack() #fill_value=5000
    .reset_index()
)

age_activity['to_rv'] = age_activity['offer_viewed'] - age_activity['offer_received']
age_activity['to_vc'] = age_activity['offer_completed'] - age_activity['offer_viewed']
age_activity['to_rc'] = age_activity['offer_completed'] - age_activity['offer_received']


age_activity.round(1)

In [ ]:
df = person_activity[(person_activity['offer_id'] == 'ofr_A')]

sns.histplot(df['offer_completed'], bins=30, kde=False, color='blue', label='To VR')
#sns.histplot(df['offer_received'], bins=30, kde=True, color='green', label='To CV')
#sns.histplot(df['offer_viewed'], bins=30, kde=True, color='orange', label='To CR')

plt.legend()
plt.title('Time Distribution for Completing Offers')
plt.xlabel('Time (in days)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
portfolio

In [ ]:
transcript_b

In [ ]:
transcript_b.groupby(['offer_id', 'event'])['person'].count().unstack().fillna(0).reset_index()

In [ ]:
offer_influence = (
    person_activity
    .groupby(['offer_id', 'ofr_influence'])['person'].count()
    .unstack(1)
    .reset_index()
    .fillna(0)
)

offer_influence['ofr_total'] = (offer_influence[True] + offer_influence[False])
offer_influence['inf_rate'] = offer_influence[True] / (offer_influence[True] + offer_influence[False])

offer_influence

In [ ]:
offer_influence = (
    person_activity
    .groupby(['offer_id', 'extrm_ofr_influence'])['person'].count()
    .unstack(1)
    .reset_index()
    .fillna(0)
)

offer_influence['ofr_total'] = (offer_influence[True] + offer_influence[False])
offer_influence['ext_inf_rate'] = offer_influence[True] / (offer_influence[True] + offer_influence[False])

offer_influence

In [ ]:
df4 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['offer_id', 'time'])['person'].count().reset_index() #.unstack(1).fillna(0)
df4

In [ ]:
df4.plot(x='time', y='person')

In [ ]:
df = person_activity[(person_activity['offer_id'] == 'ofr_B')]

bins = int((714/24)//5)
sns.histplot(df['offer_completed'], bins=bins, kde=False, color='blue', label='To VR')
#sns.histplot(df['offer_received'], bins=30, kde=True, color='green', label='To CV')
#sns.histplot(df['offer_viewed'], bins=30, kde=True, color='orange', label='To CR')

plt.legend()
plt.title('Time Distribution for Completing Offers')
plt.xlabel('Time (in days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.scatterplot(data=person_activity, x='n_com', y='amount', hue='offer_id')
plt.title('Completion vs. Amount Spent')
plt.xlabel('Number of Offers Completed')
plt.ylabel('Total Amount Spent')
plt.show()


In [ ]:
correlation_matrix = person_activity[['n_rec', 'n_vie', 'n_com', 'amt_mean']].corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Between Metrics')
plt.show()


In [ ]:
plt.scatter(person_activity[person_activity['offer_id'] == 'ofr_A']['offer_received'], y=person_activity[person_activity['offer_id'] == 'ofr_A']['amount'])
plt.scatter(person_activity[person_activity['offer_id'] == 'ofr_A']['offer_viewed'], y=person_activity[person_activity['offer_id'] == 'ofr_A']['amount'])
plt.scatter(person_activity[person_activity['offer_id'] == 'ofr_A']['offer_completed'], y=person_activity[person_activity['offer_id'] == 'ofr_A']['amount'])

In [ ]:
grouped1 = transcript_b[transcript_b['event'] == 'offer_completed'].groupby(['person', 'offer_id'])
grouped2 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['person', 'offer_id'])

aggregations1 = grouped1.agg(
    offer_fidelity=('event', 'count'),
    first_comp =('time', 'min'), 
    last_comp =('time', 'max'), 
).reset_index()

aggregations2 = grouped2.agg(
    fidelity_tran=('event', 'count'),
    first_time_tr=('time', 'min'),
    last_time_tr=('time', 'max')

).reset_index()

fidelity = pd.merge(
    how='left',
    left=aggregations1,
    right=aggregations2,
    on=['person']
).drop(columns=['offer_id_y'])


fidelity['recency'] = fidelity['first_time_tr'] <= 714
fidelity['recency_t1'] = fidelity['last_time_tr'] > 178.5*2
fidelity['recency_t2'] = fidelity['last_time_tr'] >= 178.5*3
fidelity['recency_t3'] = fidelity['last_time_tr'] >= 178.5*4


fidelity = fidelity.replace({True: 1, False: 0})

fidelity

In [ ]:
fidelity[(fidelity['last_time_tr'] > 178.5*2) & (fidelity['offer_id_x'] == 'ofr_A')]['person'].count()

In [ ]:
fidelity[(fidelity['last_time_tr'] > 178.5*2) & (fidelity['offer_id_x'] == 'ofr_B')]['person'].count()

In [ ]:
fidelity[(fidelity['last_time_tr'] > 178.5*2) & (fidelity['offer_id_x'] == 'ofr_C')]['person'].count()

In [ ]:
(178.5*4)/24

In [ ]:
fidelity[fidelity['recency_t3'] == 1]

In [ ]:
fidelity['recency'].sum()


In [ ]:
fidelity['recency_t1'].sum()

In [ ]:
fidelity['recency_t2'].sum()

In [ ]:
fidelity['recency_t3'].sum()

In [ ]:
fidelity_offer = fidelity[fidelity['offer_id_x'] == 'ofr_A']

pd.DataFrame([fidelity_offer['recency'].sum(),
fidelity_offer['recency_t1'].sum(),
fidelity_offer['recency_t2'].sum(),
fidelity_offer['recency_t3'].sum()]).plot(kind='bar')


In [ ]:
fidelity_offer = fidelity[fidelity['offer_id_x'] == 'ofr_B']

pd.DataFrame([fidelity_offer['recency'].sum(),
fidelity_offer['recency_t1'].sum(),
fidelity_offer['recency_t2'].sum(),
fidelity_offer['recency_t3'].sum()]).plot(kind='bar')

In [ ]:
178.5*4

In [ ]:
transcript_b['time'].unique()

In [ ]:
fidelity[['first_comp']].value_counts()

In [ ]:
fidelity.info()

In [234]:
def categorize_age(age):
    if age < 25:
        return 'Young Adult'
    elif 25 <= age < 40:
        return 'Adult'
    elif 40 <= age < 60:
        return 'Middle-Aged'
    else:
        return 'Senior'

# Aplicar a função ao DataFrame
tb_transactions['age_segment'] = tb_transactions['age'].apply(categorize_age)

In [ ]:
person_activity.dropna(axis=1)

In [38]:
df = person_activity
df = df.dropna()

X = df.drop(columns = ['person', 'event', 'became_member_on', 'gender', 'income'])
y = df['gender']

In [ ]:
y.value_counts()

In [ ]:
model1 = RandomForestClassifier(
    random_state=42,
    max_depth=15,
    n_estimators=25,
    min_samples_leaf=2,
    min_samples_split=2,
    #max_features=0.5,
    n_jobs=-1    
    )

model2 = DecisionTreeClassifier(
    random_state=42,
    criterion='entropy',            # Função de impureza para divisão ('gini' ou 'entropy')
    max_depth=15,                # Profundidade máxima da árvore (controle de overfitting)
    min_samples_split=2,         # Número mínimo de amostras para dividir um nó
    min_samples_leaf=2,          # Número mínimo de amostras em uma folha
    #max_features=5,           # Número máximo de features consideradas em cada divisão
    #splitter='random'            # Estratégia para escolher a divisão ('best' ou 'random')
    )



In [ ]:
model_type = model2
train_model(X, y, model_type)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Instanciar o modelo
rf = RandomForestClassifier(random_state=42)

# Definir a grade de parâmetros
param_grid = {
    'n_estimators': [50, 100, 200],        # Número de árvores na floresta
    'max_depth': [None, 10, 20, 30],       # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],       # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],         # Número mínimo de amostras em uma folha
    'max_features': ['sqrt', 'log2'],      # Número de recursos considerados para divisão
    'bootstrap': [True, False]             # Amostragem com ou sem reposição
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                  # Validação cruzada (5-fold)
    scoring='accuracy',    # Métrica de avaliação
    verbose=2,             # Nível de detalhe das mensagens
    n_jobs=-1              # Paralelização (usar todos os núcleos disponíveis)
)

# Ajustar o modelo aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e a melhor pontuação
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor pontuação (treino):", grid_search.best_score_)

# Avaliar o modelo com os melhores parâmetros nos dados de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Acurácia nos dados de teste:", test_accuracy)


y_pred_train = best_model.predict(X_train)
y_pred = best_model.predict(X_test)

# 1. Confusion Matrix
cm1 = confusion_matrix(y_train, y_pred_train)
cm2 = confusion_matrix(y_test, y_pred)


# 2. Relatório com Precision, Recall e F1-Score
report1 = classification_report(y_test, y_pred, target_names=best_model.classes_, zero_division=1)
print("Test Classification Report:")
print(report1)

report2 = classification_report(y_train, y_pred_train, target_names=best_model.classes_, zero_division=1)
print("Train Classification Report:")
print(report2)